### Importation des bibilothéques

In [ ]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend


### Définition de la fonction d'activation personnalisée :

In [ ]:
# custom activation function
def custom_activation(output):
    logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result


### Définition des modèles de discriminateur supervisé et non supervisé :

In [ ]:
# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(28,28,1), n_classes=10):
 # image input
 in_image = Input(shape=in_shape)
 # downsample
 fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
 fe = LeakyReLU(alpha=0.2)(fe)
 # downsample
 fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
 fe = LeakyReLU(alpha=0.2)(fe)
 # downsample
 fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
 fe = LeakyReLU(alpha=0.2)(fe)
 # flatten feature maps
 fe = Flatten()(fe)
 # dropout
 fe = Dropout(0.4)(fe)
 # output layer nodes
 fe = Dense(n_classes)(fe)
 # supervised output
 c_out_layer = Activation('softmax')(fe)
 # define and compile supervised discriminator model
 c_model = Model(in_image, c_out_layer)
 c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
 # unsupervised output
 d_out_layer = Lambda(custom_activation)(fe)
 # define and compile unsupervised discriminator model
 d_model = Model(in_image, d_out_layer)
 d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
 return d_model, c_model

### Définition du modèle de générateur

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
 # image generator input
 in_lat = Input(shape=(latent_dim,))
 # foundation for 7x7 image
 n_nodes = 128 * 7 * 7
 gen = Dense(n_nodes)(in_lat)
 gen = LeakyReLU(alpha=0.2)(gen)
 gen = Reshape((7, 7, 128))(gen)
 # upsample to 14x14
 gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
 gen = LeakyReLU(alpha=0.2)(gen)
 # upsample to 28x28
 gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
 gen = LeakyReLU(alpha=0.2)(gen)
 # output
 out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
 # define model
 model = Model(in_lat, out_layer)
 return model

### Définition du modèle GAN combiné

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
 # make weights in the discriminator not trainable
 d_model.trainable = False
 # connect image output from generator as input to discriminator
 gan_output = d_model(g_model.output)
 # define gan model as taking noise and outputting a classification
 model = Model(g_model.input, gan_output)
 # compile model
 opt = Adam(lr=0.0002, beta_1=0.5)
 model.compile(loss='binary_crossentropy', optimizer=opt)
 return model

### Chargement des données réelles

In [ ]:
# load the images
def load_real_samples():
 # load dataset
 (trainX, trainy), (_, _) = load_data()
 # expand to 3d, e.g. add channels
 X = expand_dims(trainX, axis=-1)
 # convert from ints to floats
 X = X.astype('float32')
 # scale from [0,255] to [-1,1]
 X = (X - 127.5) / 127.5
 print(X.shape, trainy.shape)
 return [X, trainy]

### Sélection d'un sous-ensemble supervisé du jeu de données

In [ ]:
# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=10):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)

    for i in range(n_classes):
        # get all images for this class
        X_with_class = X[y == i]
        # choose random instances
        ix = randint(0, len(X_with_class), n_per_class)
        # add to list
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]

    return asarray(X_list), asarray(y_list)


### Génération d'échantillons réels

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
 # split into images and labels
 images, labels = dataset
 # choose random instances
 ix = randint(0, images.shape[0], n_samples)
 # select images and labels
 X, labels = images[ix], labels[ix]
 # generate class labels
 y = ones((n_samples, 1))
 return [X, labels], y

### Génération de points dans l'espace latent

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
 # generate points in the latent space
 z_input = randn(latent_dim * n_samples)
 # reshape into a batch of inputs for the network
 z_input = z_input.reshape(n_samples, latent_dim)
 return z_input

### Utilisation du générateur pour générer des fake labels

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
 # generate points in latent space
 z_input = generate_latent_points(latent_dim, n_samples)
 # predict outputs
 images = generator.predict(z_input)
 # create class labels
 y = zeros((n_samples, 1))
 return images, y

#### Definition de fonction de calcul de performances du model

In [ ]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(100):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename1 = 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # evaluate the classifier model
    X, y = dataset
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('Classifier Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = 'g_model_%04d.h5' % (step+1)
    g_model.save(filename2)
    # save the classifier model
    filename3 = 'c_model_%04d.h5' % (step+1)
    c_model.save(filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))


####Entraînement du Modèle SGAN

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=3, n_batch=100):
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    # manually enumerate epochs
    for i in range(n_steps):
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
        # evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, c_model, latent_dim, dataset)
        if i == 2000:
           break


In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator models
d_model, c_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (60000,)
(100, 28, 28, 1) (100,)
n_epochs=3, n_batch=100, 1/2=50, b/e=600, steps=1800
2/2 [==============================] - 0s 104ms/step
>1, c[2.300,6], d[0.094,2.408], g[0.095]
2/2 [==============================] - 0s 4ms/step
>2, c[2.291,8], d[0.078,2.436], g[0.095]
2/2 [==============================] - 0s 5ms/step
>3, c[2.210,34], d[0.072,2.428], g[0.099]
2/2 [==============================] - 0s 5ms/step
>4, c[2.239,20], d[0.073,2.370], g[0.103]
2/2 [==============================] - 0s 5ms/step
>5, c[2.072,54], d[0.068,2.370], g[0.101]
2/2 [==============================] - 0s 5ms/step
>6, c[1.963,16], d[0.041,2.427], g[0.099]
2/2 [==============================] - 0s 5ms/step
>7, c[1.760,38], d[0.020,2.421], g[0.113]
2/2 [==============================] - 0s 5ms/step
>8, c[1.652,32], d[0.014,2.297], g[0.120]
2/2 [==============================] - 0s 5ms/step
>9, c[1.652,36], d[0.010,2.242], g[

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Classifier Accuracy: 93.027%
>Saved: generated_plot_0600.png, g_model_0600.h5, and c_model_0600.h5
2/2 [==============================] - 0s 6ms/step
>601, c[0.006,100], d[0.605,0.659], g[1.380]
2/2 [==============================] - 0s 4ms/step
>602, c[0.010,100], d[0.647,0.747], g[1.471]
2/2 [==============================] - 0s 4ms/step
>603, c[0.014,100], d[0.781,0.744], g[1.372]
2/2 [==============================] - 0s 6ms/step
>604, c[0.016,100], d[0.689,0.579], g[1.165]
2/2 [==============================] - 0s 8ms/step
>605, c[0.007,100], d[0.481,0.694], g[1.323]
2/2 [==============================] - 0s 8ms/step
>606, c[0.007,100], d[0.629,0.579], g[1.350]
2/2 [==============================] - 0s 4ms/step
>607, c[0.007,100], d[0.635,0.760], g[1.408]
2/2 [==============================] - 0s 4ms/step
>608, c[0.007,100], d[0.704,0.673], g[1.475]
2/2 [==============================] - 0s 8ms/step
>609, c[0.007,100], d[0.652,0.447], g[1.536]
2/2 [==============================] 

Classifier Accuracy: 93.120%
>Saved: generated_plot_1200.png, g_model_1200.h5, and c_model_1200.h5
2/2 [==============================] - 0s 6ms/step
>1201, c[0.002,100], d[0.576,0.561], g[1.329]
2/2 [==============================] - 0s 7ms/step
>1202, c[0.003,100], d[0.764,0.862], g[1.313]
2/2 [==============================] - 0s 5ms/step
>1203, c[0.005,100], d[0.529,0.693], g[1.375]
2/2 [==============================] - 0s 6ms/step
>1204, c[0.004,100], d[0.662,0.815], g[1.488]
2/2 [==============================] - 0s 6ms/step
>1205, c[0.005,100], d[0.625,0.796], g[1.815]
2/2 [==============================] - 0s 6ms/step
>1206, c[0.004,100], d[0.671,0.556], g[1.571]
2/2 [==============================] - 0s 7ms/step
>1207, c[0.003,100], d[0.886,0.601], g[1.497]
2/2 [==============================] - 0s 7ms/step
>1208, c[0.011,100], d[0.598,0.708], g[1.243]
2/2 [==============================] - 0s 5ms/step
>1209, c[0.004,100], d[0.598,1.021], g[1.134]
2/2 [=======================

Classifier Accuracy: 92.873%
>Saved: generated_plot_1800.png, g_model_1800.h5, and c_model_1800.h5


### Chargement du Modèle et Évaluation sur l'Ensemble d'Entraînement et de Test

In [ ]:
from tensorflow.keras.models import load_model
# nous Chargeon le modèle au step 1800
c_model = load_model('c_model_1800.h5')

Évaluation de la Précision sur l'Ensemble d'Entraînement et de Test et affichage des métriques par Classe dans un Rapport de Classification

In [ ]:
# example of loading the classifier model and generating images
from numpy import expand_dims
from keras.models import load_model
from keras.datasets.mnist import load_data
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

# Charger le modèle
model = load_model('c_model_1800.h5')

# load the dataset
(trainX, trainy), (testX, testy) = load_data()

# expand to 3d, e.g., add channels
trainX = expand_dims(trainX, axis=-1)
testX = expand_dims(testX, axis=-1)

# convert from ints to floats
trainX = trainX.astype('float32')
testX = testX.astype('float32')

# scale from [0,255] to [-1,1]
trainX = (trainX - 127.5) / 127.5
testX = (testX - 127.5) / 127.5

# evaluate the model
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
print('Train Accuracy: %.3f%%' % (train_acc * 100))
_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Test Accuracy: %.3f%%' % (test_acc * 100))

# make predictions
predicted_probs = model.predict(testX)
predicted_classes = np.argmax(predicted_probs, axis=1)

# Convertir les étiquettes one-hot en classes
testy_classes = np.argmax(testy, axis=1) if len(testy.shape) > 1 else testy

# calculate metrics for each class
precision_per_class = precision_score(testy_classes, predicted_classes, average=None)
recall_per_class = recall_score(testy_classes, predicted_classes, average=None)
f1_per_class = f1_score(testy_classes, predicted_classes, average=None)

# print metrics for each class
for i in range(len(precision_per_class)):
    print(f'Class {i}:')
    print(f'  Precision: {precision_per_class[i]:.3f}')
    print(f'  Recall: {recall_per_class[i]:.3f}')
    print(f'  F1-score: {f1_per_class[i]:.3f}')
    print()

# display classification report
print('Classification Report:')
print(classification_report(testy_classes, predicted_classes))


Train Accuracy: 92.873%
Test Accuracy: 93.360%
313/313 [==============================] - 1s 2ms/step
Class 0:
  Precision: 0.982
  Recall: 0.908
  F1-score: 0.944

Class 1:
  Precision: 0.987
  Recall: 0.986
  F1-score: 0.986

Class 2:
  Precision: 0.939
  Recall: 0.888
  F1-score: 0.912

Class 3:
  Precision: 0.958
  Recall: 0.931
  F1-score: 0.944

Class 4:
  Precision: 0.948
  Recall: 0.962
  F1-score: 0.955

Class 5:
  Precision: 0.899
  Recall: 0.959
  F1-score: 0.928

Class 6:
  Precision: 0.964
  Recall: 0.967
  F1-score: 0.965

Class 7:
  Precision: 0.828
  Recall: 0.956
  F1-score: 0.888

Class 8:
  Precision: 0.911
  Recall: 0.928
  F1-score: 0.920

Class 9:
  Precision: 0.938
  Recall: 0.850
  F1-score: 0.892

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       980
           1       0.99      0.99      0.99      1135
           2       0.94      0.89      0.91      1032
           3       0.96     

En partant de seulement 100 données labellisées, nous constatons que malgré la limitation en termes de données labelisé, le modèle SGAN parvient à atteindre une précision de 93%, surpassant significativement le modèle UDA qui a atteint une précision de 78%. Cela souligne l'efficacité du modèle SGAN dans l'utilisation efficace des données labelisés disponibles, démontrant ainsi sa capacité à tirer le meilleur parti d'un ensemble de données limité tout en améliorant les performances de classification.